In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [2]:
df = pd.read_csv('input/train.csv')
# df = pd.read_csv('input/train_min.csv')  # small data

fix_data_skew = False

if fix_data_skew:
    trues = df.loc[df['target'] == 1]
    falses = df.loc[df['target'] != 1].sample(frac=1)[:len(trues)]
    data = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    data = df
    
data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [3]:
X, y = data.iloc[:,2:].values, data.iloc[:,1].values

# std scaling
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [5]:
test = pd.read_csv('input/test.csv')
test_ids, test_x = test.iloc[:,0], test.iloc[:,1:]

test_x_scaled = scaler.transform(test_x)

In [105]:
# stacking

models = [
    ('lg', SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.3)),
    ('mlp', MLPClassifier(solver='lbfgs', alpha=0.001, hidden_layer_sizes=(5, 2), random_state=1)),
    ('rf', RandomForestClassifier(n_estimators=100, criterion='entropy')),
    ('gnb', GaussianNB()),
    ('qda', QuadraticDiscriminantAnalysis(tol=1e-12)),
]

cv_out = pd.DataFrame(index=data.iloc[:,0])
test_out = pd.DataFrame(index=test_ids)

# add target to cv_out
cv_out['target'] = data.iloc[:,1].values

for model in models:
    name = model[0]
    cls = model[1]
    print('working on ' + name)
    cls.fit(X_scaled, y)
    cv_y = cls.predict_proba(X_scaled)
    test_y = cls.predict_proba(test_x_scaled)
    
    cv_out[name] = pd.DataFrame(cv_y).iloc[:,1].values
    test_out[name] = pd.DataFrame(test_y).iloc[:,1].values

working on lg
working on mlp
working on rf
working on gnb
working on qda


In [106]:
# add statistics
num = len(models);

cv_out['mean'] = cv_out.iloc[:,[1,num]].mean(axis=1)
cv_out['min'] = cv_out.iloc[:,[1,num]].min(axis=1)
cv_out['max'] = cv_out.iloc[:,[1,num]].max(axis=1)

test_out['mean'] = test_out.iloc[:,[0,num-1]].mean(axis=1)
test_out['min'] = test_out.iloc[:,[0,num-1]].min(axis=1)
test_out['max'] = test_out.iloc[:,[0,num-1]].max(axis=1)

In [107]:
cv_out.head()

,target,lg,mlp,rf,gnb,qda,mean,min,max
ID_code,,,,,,,,,
train_0,0,0.081320,0.038493,0.03,0.011492,0.014285,0.047803,0.014285,0.081320
train_1,0,0.177724,0.539987,0.03,0.784894,0.861078,0.519401,0.177724,0.861078
train_2,0,0.090741,0.050064,0.02,0.007235,0.006782,0.048762,0.006782,0.090741
train_3,0,0.133575,0.176150,0.06,0.229756,0.138297,0.135936,0.133575,0.138297
train_4,0,0.115880,0.124300,0.07,0.051565,0.014879,0.065379,0.014879,0.115880


In [108]:
test_out.head()

,lg,mlp,rf,gnb,qda,mean,min,max
ID_code,,,,,,,,
test_0,0.127813,0.264927,0.13,0.109484,0.504586,0.316200,0.127813,0.504586
test_1,0.134689,0.236307,0.13,0.186863,0.229932,0.182310,0.134689,0.229932
test_2,0.093286,0.092930,0.20,0.199975,0.194236,0.143761,0.093286,0.194236
test_3,0.128253,0.130471,0.14,0.109351,0.086175,0.107214,0.086175,0.128253
test_4,0.097953,0.072104,0.13,0.030222,0.023997,0.060975,0.023997,0.097953


In [109]:
cv_out.to_csv('input/train_stack.csv')

In [110]:
test_out.to_csv('input/test_stack.csv')